## 引入工具

In [56]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
# %watermark -p numpy,sklearn,pandas
# %watermark -p ipywidgets,cv2,PIL,matplotlib,plotly,netron
# %watermark -p torch,torchvision,torchaudio
# %watermark -p tensorflow,tensorboard,tflite
# %watermark -p onnx,tf2onnx,onnxruntime,tensorrt,tvm
# %matplotlib inline
# %config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

# from IPython.display import display, Markdown, HTML, IFrame, Image, Javascript
# from IPython.core.magic import register_line_cell_magic, register_line_magic, register_cell_magic
# display(HTML('<style>.container { width:%d%% !important; }</style>' % 90))

import sys, os, io, logging, time, random, math
import json, base64, requests, shutil
import argparse, shlex, signal
import numpy as np

argparse.ArgumentParser.exit = lambda *arg, **kwargs: _IGNORE_

def _IMPORT(x):
    try: 
        x = x.strip()
        if x[0] == '/':
            with open(x) as fr:
                x = fr.read()
        elif 'github' in x or 'gitee' in x:
            if x.startswith('import '):
                x = x[7:]
            if x.startswith('https://'):
                x = x[8:]
            if not x.endswith('.py'):
                x = x + '.py'
            x = x.replace('blob/main/', '').replace('blob/master/', '')
            if x.startswith('raw.githubusercontent.com'):
                uri = 'https://' + x
                x = requests.get(uri)
                if x.status_code == 200:
                    x = x.text
            elif x.startswith('github.com'):
                uri = x.replace('github.com', 'raw.githubusercontent.com')
                mod = uri.split('/')
                for s in ['main', 'master']:
                    uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[-3:])
                    x = requests.get(uri)
                    if x.status_code == 200:
                        x = x.text
                        break
            elif x.startswith('gitee.com'):
                mod = x.split('/')
                for s in ['/raw/main/', '/raw/master/']:
                    uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[3:])
                    x = requests.get(uri)
                    if x.status_code == 200:
                        x = x.text
                        break
        exec(x, globals())
    except:
        pass

def _DIR(x, dumps=True, ret=True):
    attrs = sorted([y for y in dir(x) if not y.startswith('_')])
    result = '%s: %s' % (str(type(x))[8:-2], json.dumps(attrs) if dumps else attrs)
    if ret:
        return result
    print(result)


## 解析配置

In [118]:
from utils import EVENTS
from utils import oss_put_bytes

_IMPORT('/data/nb_easy/easy_widget.py')

# RACEURL = 'http://116.85.27.115:9119'
RACEURL = 'http://10.255.0.102:9119'
MSGKEY = 'nb.zmq.repnet_tf.gamma102'

with open('schema.json', 'r') as fr:
    schema_string = fr.read()
    schema_string = schema_string.replace('zmq.repnet_tf.test', MSGKEY)
    schema_string = schema_string.replace('http://host:port', RACEURL)
    schema = json.loads(schema_string)
easy = nbeasy_schema_parse(schema, events=EVENTS, debug=True);
# https://frepai.s3.didiyunapi.com/datasets/weights/b7597c2bf766.pkl

Box(children=(Box(children=(VBox(children=(Tab(children=(VBox(children=(HBox(children=(Text(value='zmq.repnet_…

In [238]:
b = np.random.randn(64, 200)
b[:, 0]

array([-2.02950121e-01, -1.09230309e+00, -4.21627298e-01,  1.19636688e+00,
       -1.40853282e+00, -7.10892177e-01, -1.16804728e+00, -1.28390995e+00,
        9.63631499e-01,  4.77007565e-01, -1.12761489e-01, -1.51307138e+00,
       -1.82321582e+00,  1.11242318e+00, -3.64750823e-01, -1.24428505e+00,
        9.74887326e-01, -2.78016104e-01, -2.36780761e+00,  1.57479175e-01,
        8.83232099e-01,  7.93100843e-01,  9.55954042e-01, -5.35839078e-01,
        1.25487978e+00, -1.81972211e+00,  8.33019023e-01, -7.52977472e-01,
       -4.58763521e-01, -1.43864147e+00,  2.58476795e-01,  1.04346381e+00,
       -1.99575433e-01, -1.28076424e+00,  1.03049568e+00, -1.17852777e+00,
       -8.41556889e-02, -3.11363878e-01, -4.62465635e-01,  7.62511895e-01,
        4.28534581e-01,  1.67729110e+00, -7.56788322e-01, -2.45290479e-01,
       -7.91445102e-04, -7.54382656e-02, -2.08439473e+00, -1.85629372e+00,
        1.18349700e+00, -1.53338628e-01,  1.39072827e+00, -2.37639644e+00,
       -1.58882175e-01,  

In [239]:
indices = np.argsort(b, axis=0)

In [240]:
d = np.take_along_axis(b, indices, axis=0)

In [246]:
d.shape

(64, 200)

In [260]:
f = np.arange(1.0, N + 1) / N
f.shape, f

((64,),
 array([0.015625, 0.03125 , 0.046875, 0.0625  , 0.078125, 0.09375 ,
        0.109375, 0.125   , 0.140625, 0.15625 , 0.171875, 0.1875  ,
        0.203125, 0.21875 , 0.234375, 0.25    , 0.265625, 0.28125 ,
        0.296875, 0.3125  , 0.328125, 0.34375 , 0.359375, 0.375   ,
        0.390625, 0.40625 , 0.421875, 0.4375  , 0.453125, 0.46875 ,
        0.484375, 0.5     , 0.515625, 0.53125 , 0.546875, 0.5625  ,
        0.578125, 0.59375 , 0.609375, 0.625   , 0.640625, 0.65625 ,
        0.671875, 0.6875  , 0.703125, 0.71875 , 0.734375, 0.75    ,
        0.765625, 0.78125 , 0.796875, 0.8125  , 0.828125, 0.84375 ,
        0.859375, 0.875   , 0.890625, 0.90625 , 0.921875, 0.9375  ,
        0.953125, 0.96875 , 0.984375, 1.      ]))

In [258]:
f.repeat(200, axis=0).shape

(12800,)

In [283]:
g1 = g[:10]

In [300]:
g2 = g[10:20] * 1.7

In [309]:
np.abs(g2 - g1).max(axis=0)

array([0.43876165, 0.30013266, 1.3091773 , 0.9663438 , 0.48876681,
       0.61970258, 0.50867547, 0.53677501, 0.73527378, 0.45758431,
       0.59092134, 0.34994418, 0.24895333, 0.29230571, 0.33076451,
       0.39739934, 0.42299426, 0.68795045, 0.54862245, 0.28582469,
       0.33808172, 0.58452948, 0.25550049, 0.49040126, 0.60429707,
       1.30503606, 0.31639731, 0.6473854 , 0.48977211, 0.49061136,
       0.55138459, 0.5527943 , 0.5653104 , 0.21761376, 0.45176667,
       0.5860543 , 0.28852019, 0.40876617, 1.02093548, 0.32466637,
       0.77743779, 0.31157242, 0.32706439, 0.69922151, 1.23730719,
       0.77966089, 0.90328775, 0.71560116, 0.2695922 , 0.58895516,
       0.63091766, 0.65711314, 0.40271031, 0.28921777, 0.97144508,
       0.49998382, 0.56106745, 0.7462345 , 0.44838985, 0.53599017,
       0.37779311, 0.72312428, 0.86949696, 0.49659683, 0.2878832 ,
       0.53869958, 0.68094682, 0.91541386, 0.37046539, 0.58353359,
       0.38150058, 0.34115846, 0.68216592, 0.30126758, 0.39866

In [301]:
g1.shape, g2.shape

((10, 200), (10, 200))

In [302]:
g1, g2

(array([[2.39202144, 2.60836554, 2.68980373, ..., 1.90690188, 1.9196734 ,
         1.75648998],
        [2.39905761, 2.36863908, 2.1924712 , ..., 1.51919914, 1.5167563 ,
         1.44373237],
        [2.13126973, 2.26304134, 2.18367836, ..., 1.47099873, 1.50787322,
         1.26486759],
        ...,
        [1.56364147, 1.83266094, 1.05155747, ..., 1.20244195, 1.10955954,
         1.05327597],
        [1.54915782, 1.58233911, 0.96331477, ..., 1.1717157 , 1.10845641,
         0.92852523],
        [1.44015995, 1.55961926, 0.83752559, ..., 1.18330061, 1.06264625,
         0.90368775]]),
 array([[2.46948671, 2.6496641 , 1.38062643, ..., 1.99654979, 1.72772031,
         1.53749029],
        [2.43403458, 2.5758192 , 1.37458881, ..., 1.99625844, 1.6492515 ,
         1.55836656],
        [2.3488097 , 2.43513303, 1.27142833, ..., 1.93723346, 1.63578295,
         1.48944196],
        ...,
        [1.7581867 , 1.87469119, 1.24421707, ..., 1.62383384, 1.52846826,
         0.91281444],
        [1.7

In [303]:
g3 = np.max([g1, g2], axis=0)

In [304]:
g3

array([[2.46948671, 2.6496641 , 2.68980373, ..., 1.99654979, 1.9196734 ,
        1.75648998],
       [2.43403458, 2.5758192 , 2.1924712 , ..., 1.99625844, 1.6492515 ,
        1.55836656],
       [2.3488097 , 2.43513303, 2.18367836, ..., 1.93723346, 1.63578295,
        1.48944196],
       ...,
       [1.7581867 , 1.87469119, 1.24421707, ..., 1.62383384, 1.52846826,
        1.05327597],
       [1.74087409, 1.88247177, 1.04616631, ..., 1.60301822, 1.55319649,
        0.92852523],
       [1.76288785, 1.83218482, 1.053073  , ..., 1.6212911 , 1.51536322,
        0.90368775]])

In [276]:
g = f.reshape(-1, 1) - d

In [277]:
d.shape

(64, 200)

In [266]:
d.shape

(64, 200)

In [280]:
N = 64
(g - d).max(axis=0).shape

(200,)

In [58]:
raise

RuntimeError: No active exception to reraise

1. https://frepai.s3.didiyunapi.com/datasets/vod/木槌加固/20211206123823.mp4

## Tools

In [ ]:
import numpy as np
import time
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import matplotlib.patheffects as PathEffects
from sklearn.decomposition import PCA
from scipy import stats
import pandas as pd
from sklearn import preprocessing

from matplotlib.colors import LogNorm
import itertools 
# axes = list(itertools.chain.from_iterable(axes))
from sklearn.manifold import Isomap

from scipy.signal import medfilt
import scipy.signal as signal
from statsmodels.distributions.empirical_distribution import ECDF
import pickle
import hashlib

In [ ]:
def _denormal_image(x):
    x -= x.mean()
    x /= x.std()
    x *= 64
    x += 128
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [ ]:
def outlier_detection(data):
    q1, q3 = np.percentile(data, [25, 75])
    iqr = q3 - q1
    lower_bound = q1 - (iqr * 1.5)
    upper_bound = q3 + (iqr * 1.5)
    return np.where((data < lower_bound) | (data > upper_bound))

In [ ]:
def plot_scatter(data, colors=None):
    if len(data.shape) != 2:
        xs = range(len(data))
        ys = data
    else:
        if data.shape[1] == 1:
            ys = data[:, 0]
            xs = range(len(data))
        else:
            xs = data[:, 0]
            ys = data[:, 1]
        
    plt.figure(figsize=(12, 12))
    ax = plt.subplot(aspect='equal')
    ax.scatter(xs, ys, c=colors, edgecolor='none', cmap=plt.cm.get_cmap('Paired', 2))
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)
    ax.axis('on')
    ax.axis('tight')

In [91]:
def draw_osd_sim(sim, size=128, figsize=(8, 8)):
    fig, ax = plt.subplots(figsize=figsize)
    plt.axis('off')
    # fig.set_size_inches(size / 100.0, size / 100.0)
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())
    plt.subplots_adjust(top=1, bottom=0, left=0, right=1, hspace=0, wspace=0)
    plt.margins(0,0)
    plt.imshow(sim, cmap='hot', interpolation='nearest', norm=LogNorm())
    
def draw_emb_feat(emb64):
    out64 = kspca.transform(scaler.transform(emb64))
    out64 = scaler2.fit_transform(out64).reshape((-1, 10, 10))
    fig, axes = plt.subplots(8, 8, figsize=(18, 18), sharex=True, sharey=True)
    axes = list(itertools.chain.from_iterable(axes))
    for i in range(len(axes)):
        axes[i].set_title('%.4f %.4f' % (out64[i].mean(), np.sqrt(out64[i].var())))
        axes[i].set_xticks([])
        axes[i].set_yticks([])
        axes[i].imshow(out64[i])

In [ ]:
def moving_average(arr, axis=0, wsize=3):
    return np.apply_along_axis(lambda m: np.convolve(m, np.ones(wsize)/wsize, mode='same'), axis=axis, arr=arr)

def np_info(data):
    print('%.5f' % data.min(), '%.5f' % data.max(), '%.5f' % data.mean(), '%.5f' % data.std())

In [115]:
import tensorflow_probability as tfp

ModuleNotFoundError: No module named 'tensorflow_probability'

## KSTest

In [ ]:
scaler1 = preprocessing.StandardScaler()
scaler2 = preprocessing.MinMaxScaler()
scaler3 = preprocessing.RobustScaler(quantile_range=(25., 75.))
scaler4 = preprocessing.Normalizer()
scaler5 = preprocessing.MaxAbsScaler()
NC = 200
pca = PCA(n_components=NC)
# ios = Isomap(n_components=NC)
# tril = np.tril_indices(64)
# tsne = TSNE(n_components=2, random_state=0)

In [92]:
!wget -q -P /tmp/ https://frepai.s3.didiyunapi.com/datasets/weights/b7597c2bf766.pkl

In [93]:
!wget -q -P /tmp/ https://frepai.s3.didiyunapi.com/datasets/vod/outputs/20211208164457/repnet_tf/embs_feat.npy

In [94]:
embs_feat = np.load('/tmp/embs_feat.npy')

In [95]:
embs_feat.shape

(33, 64, 512)

In [100]:
data_feat = embs_feat.copy()
data = data_feat.reshape((-1, 512))

cdata = data.copy()
scaler = scaler4

kspca = pca

# filter_func = lambda x: moving_average(x, wsize=3)
# filter_func = lambda x: signal.medfilt2d(x, kernel_size=(3,1))
# filter_func = lambda x: np.diff(x, axis=0, prepend=0)
filter_func = lambda x: x

cdata = filter_func(cdata)

scaler.fit(cdata)
data_out = kspca.fit_transform(scaler.transform(cdata))
print(data_out.shape)

ecdfs = [ECDF(sample) for sample in data_out.T]

# print(kspca.explained_variance_ratio_)

def empirical_kstest(test, ecdfs, alpha=0.01):
    pvals = np.array([stats.kstest(test[:, i], cdf=lambda x: ecdfs[i](x))[1] for i in range(test.shape[-1])])
    return sum(kspca.explained_variance_ratio_[pvals > alpha])
    # return np.mean(pvals > alpha)
    
sum(kspca.explained_variance_ratio_) / 2, sum(kspca.explained_variance_ratio_)

(2112, 200)


(0.49501621042145416, 0.9900324208429083)

In [113]:
%timeit [stats.kstest(data_out[:64][:, i], cdf=lambda x: ecdfs[i](x))[1] for i in range(100)]

313 ms ± 11.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [104]:
%%time

empirical_kstest(data_out[:64], ecdfs)

CPU times: user 628 ms, sys: 0 ns, total: 628 ms
Wall time: 625 ms


0.9832617317297263

## Save PKL

In [ ]:
ksmodel = {
    'pca': kspca,
    'scaler': scaler,
    'ecdfs': ecdfs,
}

pcaks_pkl_path = '/data/ksmodel.pkl'

with io.BytesIO() as bio:
    pickle.dump(ksmodel, bio)
    md5 = hashlib.md5(bio.getvalue()).hexdigest()[:12]
    oss_put_bytes(bio.getvalue(), f'datasets/weights/{md5}.pkl')
    print(f'https://frepai.s3.didiyunapi.com/datasets/weights/{md5}.pkl')

In [ ]:
for embs in pos_feat_list:
# for embs in test_feat_list:
    print('')
    for f in embs:
        f = filter_func(f)
        out = kspca.transform(scaler.transform(f))
        print('%.6f' % empirical_kstest(out, ecdfs), end='\t')

In [ ]:
  
# draw_osd_sim(np.concatenate(sims, axis=1), figsize=(sims.shape[0] * 4, 8))

fig, axes = plt.subplots(3, 3, figsize=(18, 18), sharex=True)
axes = list(itertools.chain.from_iterable(axes))
for i in range(len(axes)):
    axes[i].set_title('feature %d (%.6f)' % (i + 1, kspca.explained_variance_ratio_[i]))
    axes[i].plot(ecdf[i].x, ecdf[i].y, color='b', lw=3, label='expect');
    for j in range(embs.shape[0]):
        label = f'e_{j}'
        out = kspca.transform(scaler.transform(embs[j]))
        cdf = ECDF(out.T[i])
        statistic, pvalue = stats.kstest(out.T[i], cdf=lambda x: ecdf[i](x))
        axes[i].plot(cdf.x, cdf.y, lw=2, label='%s %.6f %.6f'%(label, statistic, pvalue));
    axes[i].legend();

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(18, 18), sharex=True)
axes = list(itertools.chain.from_iterable(axes))
for i in range(len(axes)):
    axes[i].set_title('feature %d (%.6f)' % (i + 1, kspca.explained_variance_ratio_[i]))
    axes[i].plot(ecdf[i].x, ecdf[i].y, color='b', lw=3, label='global');
    for label, feat in (('good', pos_feat_list[0]), ('bad', neg_feat_list[-1])):
        out = kspca.transform(scaler.transform(feat[1]))
        cdf = ECDF(out.T[i])
        statistic, pvalue = stats.kstest(out.T[i], cdf=lambda x: ecdf[i](x))
        axes[i].plot(cdf.x, cdf.y, lw=2, label='%s %.6f %.6f'%(label, statistic, pvalue));
    axes[i].legend();

In [ ]:
for feat in pos_feat_list[-2:] + neg_feat_list:
    print('')
    for f in feat[1:-1]:
        f = filter_func(f)
        out = kspca.transform(scaler.transform(f))
        print('%.6f' % empirical_kstest(out, ecdf), end='\t')
    print('')

In [ ]:
for feat in (feat_np_922, feat_np_905, feat_np_601):
    print('')
    for f in feat:
        f = filter_func(f)
        out = kspca.transform(scaler.transform(f))
        print('%.6f' % empirical_kstest(out, ecdf), end='\t')

In [ ]:
for feat in (feat_np_855, feat_np_313):
    print('')
    for f in feat:
        f = filter_func(f)
        out = kspca.transform(scaler.transform(f))
        print('%.6f' % empirical_kstest(out, ecdf), end='\t')

In [ ]:
_, ax = plt.subplots(1, 1, figsize=(12, 12))
ax.plot(
    kspca.explained_variance_ratio_, "--o", linewidth=2,
    label="Explained variance ratio"
)
ax.plot(
    kspca.explained_variance_ratio_.cumsum(), "--o", linewidth=2,
    label="Cumulative explained variance ratio"
)

ax.legend(loc="best", frameon=True);

In [ ]:
draw_osd_sim(np.concatenate(sims_np_055, axis=1), figsize=(sims_np_055.shape[0] * 4, 8))